# LiH VQE with ZNE and PE-LiNN

This tutorial builds a simple LiH VQE (6-qubit Hamiltonian), then compares:
- Noisy energy estimate
- ZNE mitigation (Mitiq)
- PE-LiNN prediction

The goal is a lightweight, reproducible baseline using the code in this repo.


In [1]:
from __future__ import annotations

import math
import random
import warnings
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset

# Ensure repo modules resolve when running from repo root or tutorials/.
ROOT = Path.cwd()
if not (ROOT / 'Experiments').exists():
    ROOT = ROOT.parent

import sys
sys.path.insert(0, str(ROOT))
sys.path.insert(0, str(ROOT / 'Experiments'))

from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer import AerSimulator
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.units import DistanceUnit

try:
    from qiskit.primitives import BackendEstimatorV2 as BackendEstimator
except Exception:
    try:
        from qiskit.primitives import BackendEstimator
    except Exception:
        try:
            from qiskit.primitives import Estimator as BackendEstimator  # type: ignore
        except Exception:
            BackendEstimator = None

from pelinn.model import PELiNNQEM, physics_loss
from pelinn.data.qiskit_dataset import circuit_features, make_noise_model, synthesize_samples, estimate_expectation

try:
    from baselines.zne_mitiq import mitigate_with_zne
except Exception as exc:
    raise RuntimeError(
        'Mitiq is required for ZNE. Install mitiq==0.48.1 with Python <3.12.'
    ) from exc


ModuleNotFoundError: No module named 'torch'

## LiH Hamiltonian (active space 4e/4o)
We build LiH at 1.6 Angstrom in STO-3G and map to qubits via the parity mapper with two-qubit reduction.


In [ ]:
# Build LiH Hamiltonian (active space 4e/4o, parity mapper with 2-qubit reduction).
atom = 'Li 0.0 0.0 0.0; H 0.0 0.0 1.6'

driver = PySCFDriver(atom=atom, unit=DistanceUnit.ANGSTROM, basis='sto3g')
problem = driver.run()
problem = ActiveSpaceTransformer(num_electrons=4, num_spatial_orbitals=4).transform(problem)

second_q_op = problem.hamiltonian.second_q_op()
mapper = ParityMapper(num_particles=problem.num_particles)
HAMILTONIAN = mapper.map(second_q_op)

print('LiH qubits:', HAMILTONIAN.num_qubits, '| terms:', len(HAMILTONIAN))


## VQE ansatz and energy evaluation helper
We use a small nsatz and optimize against the ideal (noise-free) energy.


In [ ]:
from qiskit.circuit.library import EfficientSU2
import inspect



num_qubits = HAMILTONIAN.num_qubits
ansatz = EfficientSU2(num_qubits=num_qubits, reps=1, entanglement='linear')
param_list = list(ansatz.parameters)

def build_circuit(theta: np.ndarray) -> QuantumCircuit:
    mapping = {p: float(v) for p, v in zip(param_list, theta)}
    return ansatz.assign_parameters(mapping, inplace=False)

def _is_estimator_v2(estimator) -> bool:
    try:
        return "pubs" in inspect.signature(estimator.run).parameters
    except (TypeError, ValueError):
        return False

def estimate_energy(
    qc: QuantumCircuit,
    observable: SparsePauliOp,
    shots: int | None = None,
    noise_cfg: dict | None = None,
    return_var: bool = False,
) -> float | tuple[float, float]:
    if BackendEstimator is None:
        raise RuntimeError("BackendEstimator/Estimator is required for non-Z observables.")
    noise_model = make_noise_model(**noise_cfg) if noise_cfg else None
    backend = AerSimulator(noise_model=noise_model)
    estimator = BackendEstimator(backend=backend)
    compiled = transpile(qc, backend)

    if _is_estimator_v2(estimator):
        pub = (compiled, observable)
        precision = None
        if shots is not None and shots > 0:
            precision = 1.0 / math.sqrt(shots)
        res = estimator.run([pub], precision=precision).result()
        pub_res = res[0]
        value = float(pub_res.data.evs)
        if return_var:
            stds = getattr(pub_res.data, "stds", None)
            if stds is None:
                var = 0.0
            else:
                var = float(np.asarray(stds)) ** 2
            return value, var
        return value

    res = estimator.run(circuits=compiled, observables=observable, shots=shots).result()
    try:
        value = float(res.values[0])
    except Exception:
        value = float(res[0])
    if return_var:
        var = 0.0
        try:
            meta0 = res.metadata[0]
            if isinstance(meta0, dict):
                var = float(meta0.get("variance", 0.0))
            else:
                var = float(getattr(meta0, "variance", 0.0))
        except Exception:
            var = 0.0
        return value, var
    return value

def ideal_energy(theta: np.ndarray) -> float:
    qc = build_circuit(theta)
    return estimate_energy(qc, HAMILTONIAN, shots=None, noise_cfg=None)


In [ ]:
# Simple VQE optimization (ideal backend).
try:
    from scipy.optimize import minimize
except Exception:
    minimize = None

rng = np.random.default_rng(1234)
theta0 = rng.uniform(0, 2 * math.pi, size=len(param_list))

if minimize is None:
    # Fallback: random search
    best_theta = theta0
    best_val = ideal_energy(theta0)
    for _ in range(30):
        trial = rng.uniform(0, 2 * math.pi, size=len(param_list))
        val = ideal_energy(trial)
        if val < best_val:
            best_val = val
            best_theta = trial
    theta_opt = best_theta
    energy_opt = best_val
else:
    res = minimize(ideal_energy, theta0, method='COBYLA', options={'maxiter': 60})
    theta_opt = res.x
    energy_opt = res.fun

print('Ideal VQE energy:', energy_opt)


## Noisy energy and ZNE mitigation
We evaluate the optimized circuit under a configurable noise model, then apply ZNE.


In [ ]:
noise_cfg = {
    'p1_depol': 0.005,
    'p2_depol': 0.02,
    'p_amp': 0.005,
    'readout_p01': 0.02,
    'readout_p10': 0.02,
}
shots_noisy = 512

qc_opt = build_circuit(theta_opt)

energy_noisy = estimate_energy(qc_opt, HAMILTONIAN, shots=shots_noisy, noise_cfg=noise_cfg)
print('Noisy energy (optimized circuit):', energy_noisy)

def noisy_executor(circuits):
    if isinstance(circuits, (list, tuple)):
        return [estimate_energy(qc, HAMILTONIAN, shots=shots_noisy, noise_cfg=noise_cfg) for qc in circuits]
    return estimate_energy(circuits, HAMILTONIAN, shots=shots_noisy, noise_cfg=noise_cfg)

noisy_executor.__annotations__["return"] = float


## Train PE-LiNN on mixed VQE circuits
We train PE-LiNN on noisy/ideal pairs generated from a mix of random ansatz parameters
and a local neighborhood around the optimized parameters. This reduces distribution shift
when evaluating near the VQE optimum.


In [ ]:
class SamplesDataset(Dataset):
    def __init__(self, samples, mean=None, std=None):
        self.X = torch.tensor(np.stack([s.x for s in samples]), dtype=torch.float32)
        if mean is not None and std is not None:
            mean_t = torch.tensor(mean, dtype=torch.float32).to(self.X)
            std_t = torch.tensor(std, dtype=torch.float32).to(self.X)
            self.X = (self.X - mean_t) / std_t
        self.y = torch.tensor([s.y_ideal for s in samples], dtype=torch.float32)
        self.cid = torch.tensor([int(s.meta.get('circuit_index', i)) for i, s in enumerate(samples)], dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], self.cid[idx]

def make_groups(cids):
    grouped = {}
    for idx, cid in enumerate(cids):
        grouped.setdefault(int(cid), []).append(idx)
    return list(grouped.values())

def compute_feature_stats(samples):
    feats = np.stack([s.x for s in samples]).astype(np.float32)
    mean = feats.mean(axis=0)
    std = np.maximum(feats.std(axis=0), 1e-6)
    return mean, std

# Build training circuits (mix of random + local around theta_opt).
num_train_circuits = 600
num_local = int(num_train_circuits * 0.4)
num_global = num_train_circuits - num_local

theta_global = rng.uniform(0, 2 * math.pi, size=(num_global, len(param_list)))
theta_local = theta_opt + rng.normal(0, 0.25, size=(num_local, len(param_list)))
theta_local = np.mod(theta_local, 2 * math.pi)
theta_train = np.vstack([theta_global, theta_local])
rng.shuffle(theta_train)

train_circuits = [build_circuit(theta) for theta in theta_train]
train_observables = [HAMILTONIAN for _ in train_circuits]

noise_grid = [
    noise_cfg,
    {
        'p1_depol': 0.01,
        'p2_depol': 0.03,
        'p_amp': 0.01,
        'readout_p01': 0.03,
        'readout_p10': 0.03,
    },
]

samples = synthesize_samples(
    train_circuits,
    train_observables,
    noise_grid,
    shots_noisy=shots_noisy,
    shots_ideal=0,
)

feature_mean, feature_std = compute_feature_stats(samples)
train_dataset = SamplesDataset(samples, mean=feature_mean, std=feature_std)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = PELiNNQEM(
    in_dim=train_dataset.X.shape[1],
    hid_dim=96,
    steps=6,
    dt=0.25,
    use_tanh_head=False,
).to(device)

optimiser = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-2)

epochs = 60
for epoch in range(1, epochs + 1):
    model.train()
    running = 0.0
    for X, y, cid in train_loader:
        X = X.to(device)
        y = y.to(device)
        preds = model(X)
        loss = physics_loss(
            preds,
            y,
            make_groups(cid.tolist()),
            alpha_inv=0.1,
            reg_gate=model.last_gate_reg,
            reg_A=model.last_A_reg,
        )
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        running += float(loss.detach())
    if epoch in {1, epochs} or epoch % 10 == 0:
        print(f"Epoch {epoch}/{epochs} | loss={running / max(1, len(train_loader)):.6f}")


## PE-LiNN prediction for the optimized VQE circuit
We compute the same feature vector used during training and predict the mitigated energy.


In [ ]:
import pandas as pd

# Evaluate on a batch of circuits near the optimum.
num_test_circuits = 25
theta_test = theta_opt + rng.normal(0, 0.15, size=(num_test_circuits, len(param_list)))
theta_test[0] = theta_opt
theta_test = np.mod(theta_test, 2 * math.pi)
test_circuits = [build_circuit(theta) for theta in theta_test]

ideal_vals = np.array([estimate_energy(qc, HAMILTONIAN, shots=None, noise_cfg=None) for qc in test_circuits], dtype=np.float64)

noisy_vals = []
zne_vals = []
pelinn_vals = []

for qc in test_circuits:
    y_noisy, var_noisy = estimate_energy(qc, HAMILTONIAN, shots=shots_noisy, noise_cfg=noise_cfg, return_var=True)
    noisy_vals.append(y_noisy)
    zne_vals.append(
        mitigate_with_zne(
            executor=noisy_executor,
            circuit=qc,
            observable=None,
            scale_factors=(1.0, 3.0, 5.0),
        )
    )
    feat = circuit_features(qc).astype(np.float32)
    noise_vec = np.array([
        noise_cfg['p1_depol'],
        noise_cfg['p2_depol'],
        noise_cfg['p_amp'],
        noise_cfg['readout_p01'],
        noise_cfg['readout_p10'],
    ], dtype=np.float32)
    x = np.concatenate([feat, noise_vec, np.array([y_noisy, var_noisy, shots_noisy], dtype=np.float32)])
    x = (x - feature_mean) / feature_std
    model.eval()
    with torch.no_grad():
        pred = float(model(torch.tensor(x, dtype=torch.float32).unsqueeze(0).to(device)).cpu().item())
    pelinn_vals.append(pred)

noisy_vals = np.array(noisy_vals, dtype=np.float64)
zne_vals = np.array(zne_vals, dtype=np.float64)
pelinn_vals = np.array(pelinn_vals, dtype=np.float64)

def metrics(y_true, y_pred):
    diff = y_pred - y_true
    return {
        "mae": float(np.mean(np.abs(diff))),
        "rmse": float(np.sqrt(np.mean(diff ** 2))),
        "max_abs": float(np.max(np.abs(diff))),
    }

summary = [
    ("NOISY", metrics(ideal_vals, noisy_vals)),
    ("ZNE", metrics(ideal_vals, zne_vals)),
    ("PE-LINN", metrics(ideal_vals, pelinn_vals)),
]
summary_df = pd.DataFrame(
    [(name, m["mae"], m["rmse"], m["max_abs"]) for name, m in summary],
    columns=["method", "mae", "rmse", "max_abs"],
)
summary_df
